In [1]:
from __future__ import division
import datetime as dt
from collections import OrderedDict
import sys, os
import dateutil.relativedelta as rd
import json
from pathlib import Path

import utm
import pandas as pd
import numpy as np
import shapely.geometry as sg

DIR = Path('..')
sys.path.append(str(DIR))

import gtfstk as gt

%load_ext autoreload
%autoreload 2

DATA_DIR = DIR/'data'

In [2]:
#path = DATA_DIR/'cairns_gtfs.zip'
path = DATA_DIR/'seq_20170310_gtfs.zip'
feed = gt.read_gtfs(path, dist_units='km')

# Pick date
date = feed.get_first_week()[0]
print('date', date)


date 20170313


In [4]:
print(dt.datetime.now())
a = feed.assess_quality()
print(dt.datetime.now())
a

2017-04-14 19:36:02.988721
2017-04-14 19:36:04.929232


,indicator,value
0,num_route_short_names_duplicated,548
1,frac_route_short_names_duplicated,0.504604
2,num_stop_time_dists_missing,3370787
3,frac_stop_time_dists_missing,1
4,num_direction_ids_missing,0
5,frac_direction_ids_missing,0
6,num_trips_missing_shapes,0
7,frac_trips_missing_shapes,0
8,num_departure_times_missing,0
9,frac_departure_times_missing,0


In [ ]:
st = feed.stop_times.sort_values(['trip_id', 'stop_sequence'])
g = st.groupby('trip_id').first().reset_index()
g

In [ ]:
pd.isnull(f['transfers'].iat[2])


In [ ]:
report = feed.validate()

In [ ]:
report

In [ ]:
f = feed.stop_times.copy()
i = 10
f.ix[9:11]

In [ ]:
feed.assess()

In [ ]:
ts = feed.compute_trip_stats()
#ts.to_csv(str(DATA_DIR/'cairns_trip_stats.csv'), index=False)
ts.head()

In [ ]:
rts = feed.compute_route_time_series(ts, date)
gt.downsample(rts, freq='12H')

In [ ]:
trip_id = feed.trips['trip_id'].iat[0]
geo = feed.trip_to_geojson(trip_id, include_stops=True)
path = DATA_DIR/'trip_{!s}.geojson'.format(route_id)
with path.open('w') as tgt:
    json.dump(geo, tgt)

In [ ]:
route_id = feed.routes['route_id'].iat[0]
geo = feed.route_to_geojson(route_id, include_stops=True)
path = DATA_DIR/'route_{!s}.geojson'.format(route_id)
with path.open('w') as tgt:
    json.dump(geo, tgt)

In [ ]:
# Add distances to feed
#
trip_stats = feed.compute_trip_stats(compute_dist_from_shapes=True)
feed = feed.append_dist_to_stop_times(trip_stats)
feed.stop_times.T

In [ ]:
trip_stats = feed.compute_trip_stats()
route_stats = feed.compute_route_stats(trip_stats, date)
route_stats


In [ ]:
a = pd.Series([np.nan, np.nan])
a.dropna().max()
#gt.get_trips(feed, date, "07:30:00")


In [ ]:

# Pick screen line
path = DATA_DIR/'cairns_screen_line.geojson'
with path.open() as src:
    line = json.load(src)
    line = sg.shape(line['features'][0]['geometry'])

g = feed.compute_screen_line_counts(line, date)
g

In [ ]:
g.groupby(['route_id', 'orientation']).count()